<a href="https://colab.research.google.com/github/alifele/EEG-computation/blob/master/colab_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mat4py

In [2]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mat4py as mt
import os
from IPython.display import Image
from IPython.core.display import HTML 
from itertools import combinations
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv1D, Dropout, MaxPooling1D
from keras.utils import to_categorical
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:
!git clone https://github.com/alifele/EEG-computation.git
%cd EEG-computation

fatal: destination path 'EEG-computation' already exists and is not an empty directory.
/content/EEG-computation


In [4]:
%ls

 ali*                'EEG Data'/        Pictures/    Untitled.ipynb*
 colab_1D_CNN.ipynb   MainCode.ipynb*   README.md*


In [0]:
def data_gen():  
    disorder_dic = []
    directory = os.fsencode('EEG Data/disorder')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        disorder_dic.append(np.array(mt.loadmat('EEG Data/disorder/{}'.format(filename))[filename.split('.')[0]]))

    channels_disorder = [ [] for i in range(len(disorder_dic))]  #channels[person][channel] it will be a numpy array
    j=0;
    for person in channels_disorder:
        for i in range(19):
            person.append(disorder_dic[j][:,i])


    normal_dic = []
    directory = os.fsencode('EEG Data/normal')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        normal_dic.append(np.array(mt.loadmat('EEG Data/normal/{}'.format(filename))[filename.split('.')[0]]))

    channels_normal = [ [] for i in range(len(normal_dic))]  #channels[person][channel] it will be a numpy array
    j=0;
    for person in channels_normal:
        for i in range(19):
            person.append(normal_dic[j][:,i])

    #normal = channels_normal
    #disorder = channels_disorder

    return channels_normal, channels_disorder

In [0]:
def New_Data_Generator_N(normal):
    number_of_samples = 45
    randlist = (132 * np.random.random(number_of_samples)).astype(int)
    new_patients = list(normal[i] for i in randlist)
    
    index = [ i for i in range(number_of_samples)]
    comb = list( combinations(index, 2))
    new_data = [[] for i in range(len(comb))]
    person = 0
    for x,y in comb:
        for chan in range(19):
            new_data[person].append((new_patients[x][chan] + new_patients[y][chan] )/2)
        normal.append(new_data[person])
        person += 1
    
    return np.array(normal)

    
def New_Data_Generator_D(disorder):
    number_of_samples = 45
    randlist = (196 * np.random.random(number_of_samples)).astype(int)
    new_patients = list(disorder[i] for i in randlist)
    
    index = [ i for i in range(number_of_samples)]
    comb = list( combinations(index, 2))
    new_data = [[] for i in range(len(comb))]
    person = 0
    for x,y in comb:
        for chan in range(19):
            new_data[person].append((new_patients[x][chan] + new_patients[y][chan] )/2)
        disorder.append(new_data[person])
        person += 1
    
    return np.array(disorder)


In [0]:
normal_list, disorder_list = data_gen()


In [0]:
n_D, n_N = len(disorder_list), len(normal_list)

In [0]:
#normal = New_Data_Generator_N(normal_list)
#disorder = New_Data_Generator_D(disorder_list)

In [0]:
# the output of this function is the full data matrix and the target
#
def data_shaper(normal, disorder):
  # 0 is the disorder and the 1 is the normal
  n_N, n_D, channels, data = normal.shape[0], disorder.shape[0], normal.shape[1], normal.shape[2] 
  target = np.vstack((np.ones((n_N,1)),np.zeros((n_D,1))))
  data = np.zeros((n_N + n_D, data, channels))
  n=0
  d=0
  i=0



  for i in range(n_N + n_D):

    if i < n_N :  # for the normal patients
        data[i] = normal[n].T
        target[i] = 1
        n +=1
        i +=1
        #print('n={}'.format(n))
    else:  # for the disorder patients
        data[i] = disorder[d].T
        target[i] = 0
        d+=1
        i+=1
        #print('d={}'.format(d))





  data, target = shuffle(data, target)
  ddata = np.zeros


            
  return data, target
    

In [0]:
data , target = data_shaper( New_Data_Generator_N(normal_list), New_Data_Generator_D(disorder_list))

In [12]:
data.shape

(2308, 15360, 19)

In [0]:
model = Sequential([
    Conv1D(filters=5, kernel_size=300, activation='relu', input_shape=(15360, 19)),
    MaxPooling1D(pool_size=14),
    Conv1D(filters=4, kernel_size=70, activation='relu'),
    MaxPooling1D(pool_size=7),
    Conv1D(filters=4, kernel_size=50, activation='relu'),
    
    Flatten(),
    Dense(100, activation='relu'),
    Dense(10, activation='relu'),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid'),
    
    
])

In [0]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [28]:
import warnings
warnings.filterwarnings('ignore')model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 15061, 5)          28505     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 1075, 5)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1006, 4)           1404      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 143, 4)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 94, 4)             804       
_________________________________________________________________
flatten_3 (Flatten)          (None, 376)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)              

In [29]:
target

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]])

In [0]:

his = model.fit(data, target, epochs=10, batch_size=3)

Epoch 1/10
2308/2308 [==============================] - 13s 6ms/step - loss: 8.1922 - acc: 0.4861
Epoch 2/10
2308/2308 [==============================] - 13s 6ms/step - loss: 8.1922 - acc: 0.4861
Epoch 3/10
2308/2308 [==============================] - 13s 6ms/step - loss: 8.1922 - acc: 0.4861
Epoch 4/10
2308/2308 [==============================] - 13s 6ms/step - loss: 8.1922 - acc: 0.4861
Epoch 5/10
2308/2308 [==============================] - 13s 5ms/step - loss: 8.1922 - acc: 0.4861
Epoch 6/10
2308/2308 [==============================] - 13s 5ms/step - loss: 8.1922 - acc: 0.4861
Epoch 7/10
2308/2308 [==============================] - 13s 5ms/step - loss: 8.1922 - acc: 0.4861
Epoch 8/10
  81/2308 [>.............................] - ETA: 12s - loss: 7.6760 - acc: 0.5185

In [0]:
x = np.arange(10)
shuffle(x)

In [0]:
model.summary()

In [0]:
ind= shuffle(np.arange(132*2))

In [0]:
index = np.arange(132*2)

In [0]:
ind = shuffle(index)

In [0]:
ind

In [0]:
np.random.shuffle([1,2,3])

In [0]:
a = [1,2,3,4,5,6]
t = [2,3,1,2,3,4]
a,b = shuffle(a,t)

In [0]:
normal.shape

In [0]:
a = np.random.random((4,6,5))
b = np.random.random((8,6,5))

In [0]:
a.shape

In [0]:
b.shape

In [0]:
c = np.vstack((a,b))

In [0]:
c.shape

In [0]:
help(data_shaper)